# 第九章 符合python风格的对象
本章致力于让我们（用户）的自定义对象，与python的内置方法或标准库类型，拥有相同的能力与行为。  
**鸭子类型**：
> 当看到一只鸟走起来像鸭子、游泳起来像鸭子、叫起来也像鸭子，那么这只鸟就可以被称为鸭子。  

鸭子类型是这样的类型：它与python的内置类型拥有相同的能力，因而与内置类型无法区分。  
python允许并鼓励用户定义这样的类型。

## 区分repr与str

In [2]:
class TestBoth:
    def __init__(self, log) -> None:
        self.log=log

    def __repr__(self) -> str:
        return f"repr {self.log}"

    def __str__(self) -> str:
        return f"str {self.log}"
test = TestBoth("what a nice day!")
print(test)
print(str(test))
print(repr(test))
print(f"{test}")
print("{}".format(test))

str what a nice day!
str what a nice day!
repr what a nice day!
str what a nice day!
str what a nice day!


可以看到，在同时定义__repr__与__str__时，除了明确的repr()方法调用了__repr__外，其余几种调用都优先调用了__str__

In [4]:
class TestStr:
    def __init__(self, log) -> None:
        self.log=log

    def __str__(self) -> str:
        return f"str {self.log}"

test = TestStr("what a nice day!")
print(test)
print(str(test))
print(repr(test))
print(f"{test}")
print("{}".format(test))

str what a nice day!
str what a nice day!
str what a nice day!
str what a nice day!


如果只定义__str__，则repr的调用结果，将不是我们期望看到的可理解字符串格式。

In [5]:
class TestRepr:
    def __init__(self, log) -> None:
        self.log=log

    def __repr__(self) -> str:
        return f"repr {self.log}"

test = TestRepr("what a nice day!")
print(test)
print(str(test))
print(repr(test))
print(f"{test}")
print("{}".format(test))

repr what a nice day!
repr what a nice day!
repr what a nice day!
repr what a nice day!
repr what a nice day!


但是，如果定义了repr方法，则所有的结果都成为可理解字符串格式了。  
事实上，python的str方法是默认调用repr的而不是相反，覆写str方法将无法涵盖到repr方法。  
因此，一个好的建议是，**让str继续维持内置的默认形式，我们只覆写repr方法。**

## 定义鸭子类型
本节提供一个较好的鸭子类型，它通过实现魔术方法，让自己具备大部分内置类型的能力。包括迭代，切片，随机访问，比较，字节化。  
同时使用classmethod支持了备用的构造方法。  
说明了自定义对象的哈希，及哈希对象的不可变性。  

In [23]:
from array import array
from math import hypot


class Vector:
    # d表示浮点，以支持与bytes对象的互相转换。
    typecode = "d"
    def __init__(self, x, y) -> None:
        self._x=float(x)
        self._y=float(y)
    
    # 这将支持Vector的元组拆包、迭代与for...in..
    def __iter__(self):
        yield self._x
        yield self._y

    # 支持切片与[]访问
    def __getitem__(self, index):
        return [self._x, self._y][index]
    
    def __repr__(self) -> str:
        return "{}:{}".format(type(self).__name__, *self)

    def __str__(self) -> str:
        return str(tuple(self))

    def __eq__(self, __o: object) -> bool:
        # 是Vector类型，且两个维度的值相等.
        return isinstance(__o, type(self)) and tuple(__o)==tuple(self)

    def __abs__(self):
        return hypot(self._x, self._y)

    # 支持转换为字节序列
    def __bytes__(self):
        # array是扁平容器，内部的item格式要求是一致的，可以被bytes化。
        # bytes([ord(self.typecode)]) 貌似等同于self.typecode.encode("utf-8")
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    @classmethod
    def frombytes(cls, octets):
        # classmethod往往提供备选构造函数
        # frombytes与bytes方法一起，构成了Vector对象与bytes的互相转换
        # bytes类型是重要的，是多环境交流的桥梁。
        # 为自定义类型定义与bytes的互相转换，允许它参与网络编程，从而与其他语言编写的程序交互。
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    # 要哈希的对象，应该是不可改变的.因此使用只读器包裹不可变属性。
    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

    # 哈希。可哈希的对象一经初始化，其哈希值就不应该变化.
    def __hash__(self) -> int:
        return hash(self.x)^hash(self.y)
    
vector = Vector(3, 4)
print(vector)
print(str(vector))
print(repr(vector))
print(*vector)
print(vector[0])
print(vector[:])
for dim in vector:
    print(dim)
print(abs(vector))
print(bytes(vector))

print(Vector.frombytes(bytes(vector)))


(3.0, 4.0)
(3.0, 4.0)
Vector:3.0
3.0 4.0
3.0
[3.0, 4.0]
3.0
4.0
5.0
b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
(3.0, 4.0)


让可哈希对象的哈希值可变是危险的，来看这样一个实例：

In [20]:
class Test:
    def __init__(self, x, y) -> None:
        self.x=x
        self.y=y

    def __hash__(self) -> int:
        return hash(self.x)^hash(self.y)

dic=dict()
test = Test(1,2)
dic[test]=(1,2)
print(f"test in dict is {dic[test]}")
test.x=9999
try:
    print(f"test in dict is {dic[test]}")
except KeyError:
    print("test not in dict!")
    print(dic)

test in dict is (1, 2)
test not in dict!
{<__main__.Test object at 0x7f52f16b8130>: (1, 2)}


在修改放入字典中的对象后，它*丢失*了。我们不再能通过修改后的对象查找到字典中的值了。

## 哈希冲突
本节说明简单哈希函数的冲突情况，尽管哈希相等，但python的字典依然能区分对象之间是否不同。  
这依赖于__eq__，因此，定义__hash__时，常常也请定义__eq__，以使得散列对象的行为与你的期望相符。

In [29]:
class Vector:
    def __init__(self, x, y) -> None:
        self._x=float(x)
        self._y=float(y)

    def __iter__(self):
        yield self._x
        yield self._y

    def __eq__(self, __o: object) -> bool:
        # 是Vector类型，且两个维度的值相等.
        return isinstance(__o, type(self)) and tuple(__o)==tuple(self)

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

    # 哈希。可哈希的对象一经初始化，其哈希值就不应该变化.
    def __hash__(self) -> int:
        return hash(self.x)^hash(self.y)
        
print("="*10)
vector = Vector(3, 4)
same_vector = Vector(3, 4)
same_hash_vector = Vector(4, 3)
print(f"hash is eq? {hash(vector)==hash(same_vector)}  {hash(vector)==hash(same_hash_vector)}")
dic = dict()
dic[vector]=tuple(vector)
print(f"in dict? {vector in dic.keys()} {same_vector in dic.keys()} {same_hash_vector in dic.keys()}")
# 在字典中存放哈希相等的key
dic[same_hash_vector]=tuple(same_hash_vector)
for key in dic.keys():
    # 是相等的，python没有擅自修改被放入字典的对象的哈希值。
    print(hash(key))

hash is eq? True  True
in dict? True True False
7
7


## 虚假的private声明
python通过为变量或函数标识符前置两个下划线(\_\_)的方法将它们表示为私有的。  
但事实上，python并不提供**真正意义上**将方法或属性声明为私有的办法。  
前置两个下划线(\_\_)的标识符将被解释器理解为\_classname\_\_variableName。  
看下面一个示例：

In [4]:
class Test:
    def __init__(self) -> None:
        self.__a="a"
        self.__b="b"
    
    def __function(self):
        pass

test = Test()
print(dir(test)[:3])

['_Test__a', '_Test__b', '_Test__function']


这是一种保护措施，但无法阻止故意的恶意行为。获得test对象的函数仍然可以通过test.\_Test\_\_a=...来修改"私有"数据成员。  
因此可以说，python提供的private声明是**虚假的**。  
如果你强烈地不愿意获取test的用户修改它，那双下划线是可接受的，但多数情况下，由解释器去做这样的工作（指修改标识符命名）显得有点耍小聪明，违背了所见即所得的原则，让代码显得不那么可靠。  
因此，一个更好的建议是，**使用单下划线**。声明类似:self.\_a=...的成员。这向调用者传达了这样的信息：我不推荐你修改这样的数据成员，否则后果自负。  

## 使用__all__暴露接口
值得注意的是，单下划线声明的方法或变量不会被from package import *导入。  
在开发大型工程中，更好的做法是使用\_\_all\_\_作为暴露接口的白名单：

In [ ]:
import math # import something here

__all__=[
    "Open",
    "function_open",
]

class Open:
    def __init__(self) -> None:
        print("Open is instanced")

# 依然可以被from package import Closed引入。
class Closed:
    def __init__(self) -> None:
        print("Closed is instanced")

def function_open():
    print("open is called")

def function_close():
    print("close is called")

定义\_\_all\_\_也是一种约定的艺术，没有出现在列表中的变量是package的编写者不推荐引用的，也不会被from package import *引入。  
尽管用户依然可以通过from package import somethingClosed的方法引用，但违背了约定，就意味着要承担可能的后果。

## 使用__slots__类节省空间
python为每个实例的数据成员提供dict，使得它们能被立即索引。这是以空间换时间的常见方法。  
但如果你的程序对空间更敏感，例如要创建百万个只拥有极少数据成员的实例，使用slots就能节省大量空间。（优化在50%以上）  
声明一个元组类型的\_\_slots\_\_，在其中放入该类所有的数据成员标识符名。  
此时self或类的实例将失去类似字典的可任意赋值的能力，类将只有slots中声明的数据成员。

In [3]:
class Test:
    __slots__=("a", "b") # slots被声明为元组，因为你不期望类的数据成员命名可能变化。
    def __init__(self) -> None:
        self.a="a"
        self.b="b"
    
    def set_c(self):
        try:
            # 没有在__slots__中声明"c"时，赋值会失败。
            self.c="c"
        except AttributeError:
            print("can't set attr 'c'!")

test = Test()
print(test.a, test.b)
test.set_c()

a b
can't set attr 'c'!


注意两点：  
- slots不会被继承，子类必须重新声明自己的slots。
- 要使得类的成员可以被弱引用，需要在slots中加入\_\_weakref\_\_。

## 覆盖类属性
不被self定义，而是直接定义在类中的数据成员。类属性会被子类继承。  
python允许通过实例去访问类的数据成员，当你这么做时（通过实例去访问或覆写类属性），实际上在为该实例创建该属性。  
也就是说，覆写类属性，其实是在创建实例属性，原本的类属性没有被修改。  
除非你通过ClassName.attr=...进行修改。不建议这样做，而应该使用继承。  

In [6]:
class Test:
    class_attr_a=1
    class_attr_b="b"

    def __init__(self) -> None:
        self.instance_attr_a="instance_a"

test = Test()
print(dir(test)) # 能获得class_attr_a/b
another_test = Test()
test.class_attr_a="test1" # 修改一个实例的类成员，不会影响另外的实例。
print(f"{test.class_attr_a}  {another_test.class_attr_a}")


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'class_attr_a', 'class_attr_b', 'instance_attr_a']
test1  1


## 为继承考虑
在面向对象编程中，继承可以说是多态性表现的核心。  
在实际编码中，应该养成这样的习惯：**考虑正在编写的类将被继承的情况**。  
也就是说，要使得**方法/属性被继承或重写后依然能正常工作**。  
一个例子：

In [7]:
class Base:
    def __repr__(self) -> str:
        # 如果在这里写为return "Base...."，就很不利于继承时正常工作了。
        return type(self).__name__

class Child(Base):
    pass

base = Base()
child = Child()
print(base, child) #　子类的__repr__在很好地工作。

Base Child
